# <center>Natural Language Processing Hands-on # 1</center>
<center><span style="font-weight: bold; font-size: 1.8rem;">Representing words and sentences</span></center>

Since most of the algorithms existing out there are designed to handle numerical data, they are hardly applicable on raw texts. However, it is definitely possible to convert a text to a numerical representation.

Ideal representations should handle **semantic**, **polysemy**, **irony** and lots of other specificities of texts. Along the decades, many text representations have been introduced to handle as many specificities as possible.

In this hands-on, you will have to convert a given corpus of texts to various representations and highlight their pros / cons.

# Installation of required packages

The packages listed below should be installed. Using a virtual environment is highly recommended but not mandatory -- that is just good practice.

In [1]:
pip install -U gensim

Requirement already up-to-date: gensim in c:\logiciels\anaconda\lib\site-packages (4.1.2)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random

from pprint import pprint as pp

import gensim
import nltk
import numpy as np
import pandas as pd
import scipy as sp

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

import nltk
# Uncomment the following line to download the reuters dataset
# nltk.download('reuters')
from nltk.corpus import reuters

START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(0)
random.seed(0)

**Note**: In NLP, we often add `<START>` and `<END>` tokens to represent the beginning and end of sentences, paragraphs or documents.

# Part 0 - Exploring the dataset

The Reuters Corpus that we will use contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 categories.

Before diving into word representations, let's explore it a little bit and simply preprocess its texts to make it more suitable.

---

We will need to standardize all texts before converting anything to a numerical representations, since it will reduce the vocabulary size. Modify the following function to:

* Add the `START_TOKEN` and `END_TOKEN` at the beginning and end of each document
* Lowercase every words
* Remove the punctuation from each document

In [3]:
nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Armel\AppData\Roaming\nltk_data...


True

In [37]:
def read_corpus(category="tea"):
    """ Read files from the specified Reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """ 
    files = reuters.fileids(category)
    for f in files:
        current_text = list(reuters.words(f))
        current_text.lower()
        current_text = [START_TOKEN] + current_text + [END_TOKEN]
        
    return current_text

In [38]:
corpus = read_corpus()

In [39]:
pp(corpus[:2], compact=True)

['<START>', 'SOVIET']


# Part I - Word representations

Now that we have preprocessed our texts we can represent them using vectors, also called embeddings in this case.

*Note: the preprocessing done here is basic. We will see in another hands-on different preprocessing steps, including some suitable for frequentist approaches.*

---

Each word representation has its pros and cons: understanding them will help you in finding the best representation that suits your use case. 

As a result, you will have to implement / load and analyze the behaviour of word vectors coming from:

* Dummy encoding
* Co-occurence matrix encoding
* Pretrained GloVe encoding

## Dummy encoding

The dummy encoding consist in encoding each individual word of our corpus into a vector filled with 0 expect at a specific position where the value is 1 (equivalent to the encoding of categorical variables).

As discussed during the course, those embeddings are kind of pointless since they don't handle a single element of the ideal word representation beside being actual vectors. They are however a good starting point to play around with texts.

---

Define a function converting the words of a corpus to a set of dummy encoded vectors. Do not forget to sort your vocabulary before assigning the vectors!

In [7]:
def dummy_encode(corpus):
    """One-hot encoding of a set of texts."""
    
    return ...

In [8]:
dummy_embeddings = dummy_encode(corpus)

If you still do not believe that this representation is pointless, try finding the most similar word to "cat" using it.

## Co-occurence matrix encoding

*This section comes from Stanford's NLP hands-on*

A co-occurrence matrix counts how often things co-occur in some environment. Given some word  $w_i$  occurring in the document, we consider the context window surrounding  $w_i$ . Supposing our fixed window size is  $n$ , then this is the  $n$  preceding and  $n$  subsequent words in that document, i.e. words  $w_{i−n} … w_{i−1}$  and  $w_{i+1} … w{i+n}$ . We build a co-occurrence matrix  $M$ , which is a symmetric word-by-word matrix in which  $M_{ij}$  is the number of times  $w_j$  appears inside  $w_i$ 's window among all documents.

**Example: Co-Occurrence with Fixed Window of n=1:**

* Document 1: "all that glitters is not gold"

* Document 2: "all is well that ends well"

|        * 	| <START> 	| all 	| that 	| glitters 	| is 	| not 	| gold 	| well 	| ends 	| <END> 	|
|---------:	|--------:	|----:	|-----:	|---------:	|---:	|----:	|-----:	|-----:	|-----:	|------:	|
|  <START> 	|       0 	|   2 	|    0 	|        0 	|  0 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|      all 	|       2 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|     that 	|       0 	|   1 	|    0 	|        1 	|  0 	|   0 	|    0 	|    1 	|    1 	|     0 	|
| glitters 	|       0 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|       is 	|       0 	|   1 	|    0 	|        1 	|  0 	|   1 	|    0 	|    1 	|    0 	|     0 	|
|      not 	|       0 	|   0 	|    0 	|        0 	|  1 	|   0 	|    1 	|    0 	|    0 	|     0 	|
|     gold 	|       0 	|   0 	|    0 	|        0 	|  0 	|   1 	|    0 	|    0 	|    0 	|     1 	|
|     well 	|       0 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    1 	|     1 	|
|     ends 	|       0 	|   0 	|    1 	|        0 	|  0 	|   0 	|    0 	|    1 	|    0 	|     0 	|
|    <END> 	|       0 	|   0 	|    0 	|        0 	|  0 	|   0 	|    1 	|    1 	|    0 	|     0 	|

The rows (or columns) of this matrix provide one type of word vectors (those based on word-word co-occurrence), but the vectors will be large in general (linear in the number of distinct words in a corpus). Thus, our next step is to run dimensionality reduction. In particular, we will run *SVD* (Singular Value Decomposition), which is a kind of generalized *PCA* (Principal Components Analysis) to select the top  k  principal components.

Reducing the dimensionality of such vectors doesn't interterfere with the semantic relationship between words. Hence, *movie* will still be closer to *theater* than to *airplane*.

### Identify distinct words

Define a function that will return a list of unique words of the corpus as well as its size.

In [9]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = -1
    
    # ------------------
    # Write your implementation here.


    # ------------------

    return corpus_words, num_corpus_words

### Compute the co-occurence matrix 

Write a method that constructs a co-occurrence matrix for a certain window-size  $n$  (with a default of $4$), considering words  $n$  before and  $n$  after the word in the center of the window.

In [10]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).
    
        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.
              
              For example, if we take the document "<START> All that glitters is not gold <END>" with window size of 4,
              "All" will co-occur with "<START>", "that", "glitters", "is", and "not".
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (a symmetric numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}
    
    # ------------------
    # Write your implementation here.


    # ------------------

    return M, word2Ind

### Reduce the dimensionality of the co-occurence matrix

Construct a method that performs dimensionality reduction on the matrix to produce $k$-dimensional embeddings. Use *SVD* to take the top $k$ components and produce a new matrix of $k$-dimensional embeddings.

In our case, we will set $k=2$.

In [11]:
def reduce_to_k_dim(M, k=2):
    """ Reduce a co-occurence count matrix of dimensionality (num_corpus_words, num_corpus_words)
        to a matrix of dimensionality (num_corpus_words, k) using the following SVD function from Scikit-Learn:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
        Params:
            M (numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): co-occurence matrix of word counts
            k (int): embedding size of each word after dimension reduction
        Return:
            M_reduced (numpy matrix of shape (number of corpus words, k)): matrix of k-dimensioal word embeddings.
                    In terms of the SVD from math class, this actually returns U * S
    """    
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))
    
        # ------------------
        # Write your implementation here.
    
    
        # ------------------

    print("Done.")
    return M_reduced

---

Great! You now have fix-sized vectors that represent each words of your corpus. Let's normalize our matrix to compare our vectors easily.

In [12]:
# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced_co_occurrence, axis=1)
M_normalized = M_reduced_co_occurrence / M_lengths[:, np.newaxis] # broadcasting

NameError: name 'M_reduced_co_occurrence' is not defined

Since we are working with vectors, we can easily measure the similarity between them using the dot product. Hence, given a specific word and its related word embedding, we can easily identify its most similar words contained in the corpus!

*Note: the dot product between two vectors is bounded between -1 and 1 and the dot product of two identical vectors is equal to 1.*

Define a function that given a word $w$ identify its most similar words in your embedding space.

In [ ]:
def most_similar(word, matrix, topn=10):
    """Return the words that have the closest embedding to the queried word."""
    
    ...
    
    return ranked_words[:topn]

## GloVe encoding

Word2Vec models are predictive by essence, since it is a neural network. However, this is not the sole method to learn geometrical encodings (vectors) of words from their co-occurrence information (how frequently they appear together in large text corpora).

GloVe is a count-based model that learn their vectors by essentially doing dimensionality reduction on the co-occurrence counts matrix. Does it remind you of something? Yes, that's exactly what you did above.

Building models is time consuming. Hence, GloVe / Word2Vec models already trained on regular training sets (Wikipedia, News, etc.) are publicly shared to be reused easily.

The below code will load a Glove model trained on wikipedia and allow us inspect easily the embeddings properties.

In [ ]:
import gensim.downloader as api

In [ ]:
# List all pretrained models available on gensim*
pp(list(api.info()["models"].keys()))

In [ ]:
model = api.load('glove-wiki-gigaword-50')

You can get the most similar embeddings to those of a given set of words. 
Here, we retrieve the most similar words to fox, rabbit and cat.

In [ ]:
pp(model.most_similar(positive=["fox", "rabbit", "cat"]), compact=True)

You can also perform concept additions / soustractions. For instance, you can 

In [ ]:
model.most_similar(positive=["king", "woman"], negative=["man"])

Explore those embeddings and comment on how they help to identify / handle:

* Synonyms
* Antonyms
* Grammatical errors
* Polysemy
* Irony
* Analogies

---

Where do you think those biases come from?

# Part II - Sentence representations

## TF-IDF

The TF-IDF is a methodology aiming at finding the most significative words in each document by comparing their in-document frequency to the overall frequency of that term in the whole corpus.

---

Convert the reuters corpus (at least one category) to its TF-IDF representation using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Beware of all the parameters of the methods! They might have significant impact on what you're doing.

Once you have converted your corpus using the TF-IDF methodology, create a function identifying the most relevant comments given a search query.

In [ ]:
def search_corpus(corpus, search_query, topn=10):
    """Retrieve the top n documents matching a search query within a list of texts.
    
    """
    
    tfidf_corpus = TfidfVectorizer(...)
    
    ...
    
    return ranked_articles[:topn]

## Unsupervised Random Walk Sentence Embeddings

This approach has been presented by Kawin Ethayarajh in 2018. The key idea behind this methodology is to take a weighted average of previously trained word embeddings and modify it with SVD (Singular Value Decomposition, a kind of generalization of the PCA).

By having a look at [this implementation](https://github.com/kawine/usif), try to compute the uSIF embeddings of our corpus and compare their properties to the TF-IDF / Averaged Word2Vec ones. 

Consider digging in the [related paper](https://aclanthology.org/W18-3012.pdf) if you want to know more about the methodology.